**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    member 1
    
    member 2
    
    ...

**Required Readings:** 
* Chapter 1 and Chapter 9 of the book [Mining the Social Web](http://www.learndatasci.com/wp-content/uploads/2015/08/Mining-the-Social-Web-2nd-Edition.pdf) 
* The codes for [Chapter 1](http://bit.ly/1qCtMrr) and [Chapter 9](http://bit.ly/1u7eP33)


** NOTE **
* Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost.

*----------------------

#Case Study 1 : Collecting Data from Twitter

** Due Date: Sep. 23, before the class**

*------------

#Problem 1: Sampling Twitter Data with Streaming API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [29]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import twitter, io,json
from prettytable import PrettyTable
import sys
import time
from urllib2 import URLError
from httplib import BadStatusLine
from functools import partial
from sys import maxint
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    
    CONSUMER_KEY = 't6Q4gXf5S4BzCk5un16Wuovld'
    CONSUMER_SECRET ='mTDPhFhCV85F2yy6Dj4fae9ItsGhtMuEX9tr6HF144IaWRqO07'
    OAUTH_TOKEN = '3602884696-Ws75LsoF1z54KCU1KW88p6eaeTaw4rpMPNSBCRr'
    OAUTH_TOKEN_SECRET = 'diuXUz3R0DRL1oRRG6qmJqsPAP7PbLI3yj6oJI63zWxZw'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

#---------------------------------------------
# Define a Function to get tweets with a limited number
def twitter_search(twitter_api, word, max_results = 8000, **kw):
    search_results = twitter_api.search.tweets(q = word, count = 100, **kw)
    statuses = search_results["statuses"]
    
    max_results = max(max_results, 8000) # set the cap to be the bigger one of the two
    
    for _ in range(80): # 80*100 = 8000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError, e: # No more results when next_results doesn't exist
            break
        nwords = dict([k.split("=") for k in next_results[1:].split("&")])
        search_results = twitter_api.search.tweets(**nwords)
        statuses += search_results["statuses"]
        if len(statuses) > max_results: #break when number of tweets exceeds limit
            break
    return statuses

#---------------------------------------------
# Define a Function to store data into a json file            
def save_json(filename, data):
    with io.open('case1/{0}.json'.format(filename), 
                 'w', encoding='utf-8') as f:
        f.write(unicode(json.dumps(data, ensure_ascii=False)))

#---------------------------------------------
# Define a Function to load data from a json file  
def load_json(filename):
    with io.open('case1/{0}.json'.format(filename), 
                 encoding='utf-8') as f:
        return f.read()

#---------------------------------------------
# Define a Function to make pretty tables. 
# The length of the label parameter should consist with that of each element in the data list
def make_table(label,data):
    pt = PrettyTable(field_names=label)
    for r in data:
        if len(r) == 0 or r[0] =='':continue
        pt.add_row(r)
    for i in label:
        pt.align[i] = 'c'# Set column alignment to Center
    print pt
    return
    
twitter_api = oauth_login()

results = twitter_search(twitter_api, "HillaryClinton") #Saving tweets about Hillary Clinton
save_json("Hillary Clinton", results)
print len(results) #Printing the total number of tweets collected


8100


# Report some statistics about the tweets you collected 

*The topic of interest: Hillary Clinton


*The total number of tweets collected:  8100

*-----------------------

#Problem 2: Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [96]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import twitter, json, re
from collections import Counter

statuses = load_json('Hillary Clinton.json') # load HC's tweets into "statuses"
#----------------------------------------------
# Define a function for word processing
# Filters all urls, user_mentions and hashtags in tweets
def word_process(tcollection):
    status_texts = [re.sub(r'(^|\s)[^\w]*[#@][^\s]*|http[s]*://[^\s]*','', s['text'],flags = re.M) for s in tcollection]
    words_origin = [w for t in status_texts for w in t.split(" ") if len(w) > 3]
    return words_origin
words_freq = Counter(word_process(statuses)) #Frequencies of the words
make_table(['Words','Count'],words_freq.most_common()[:30],30) #top 30 words with counts






+----------+-------+
|  Words   | Count |
+----------+-------+
|   that   |  597  |
|  &amp;   |  508  |
|  about   |  492  |
|   with   |  408  |
|   this   |  407  |
|  woman   |  394  |
|   says   |  383  |
|   more   |  344  |
|   have   |  342  |
| Hillary  |  311  |
|   your   |  271  |
|   than   |  255  |
|   from   |  242  |
|   real   |  239  |
|  being   |  224  |
|  first   |  211  |
|  should  |  206  |
|   just   |  202  |
|   will   |  201  |
|   like   |  192  |
|   what   |  177  |
| campaign |  174  |
|   Says   |  171  |
|  would   |  170  |
| outsider |  165  |
|   they   |  164  |
|  anyone  |  156  |
|   does   |  149  |
|   need   |  147  |
|  Trump   |  145  |
+----------+-------+


**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [19]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#----------------------------------------------
# Define a function to extract the top num of popular tweets
def popular_tweets(statuses,num):
    retweets = [(status['retweet_count'],
                 status['text'])
               for status in statuses
               if status.has_key('retweeted_status')] 
    sorted_retweets = sorted(retweets, reverse=True)[:num] # sort list in ascending order
    make_table(['Count','Popular Tweets'],sorted_retweets,num)
    return 
popular_tweets(statuses,10)









+-------+----------------------------------------------------------------------------------------------------------------------------------------------+
| Count |                                                                Popular Tweets                                                                |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------+
|  4232 | RT @TheEllenShow: Before you watch her on my show today, watch her Whip. And of course, watch her Nae Nae. @HillaryClinton https://t.co/wdk… |
|  2818 |                                       RT @HillaryClinton: Retweet if you agree. http://t.co/OjW9FBzlfm                                       |
|  2751 |                               RT @TheEllenShow: .@HillaryClinton is on my NYC premiere. http://t.co/q4k6N9NBv8                               |
|  2751 |                               RT @TheEllenShow: .@HillaryClinton is on m

**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [26]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#----------------------------------------------
# Define a function to extract hashtags or user_mentions
def popular_entities(statuses,type = "hashtags"):
    result = []
    if (type=="hashtags"):
        result = [ hashtag['text']
            for status in statuses
            for hashtag in status['entities']['hashtags'] ]
    else:
        result = [u['name'] for s in statuses for u in s['entities'][type]]
    return result
pop_user = Counter(popular_entities(statuses,"user_mentions")).most_common()[:10]
make_table(['User mentions','Count'],pop_user,10)
pop_hashtag = Counter(popular_entities(statuses,"hashtags")).most_common()[:10]
make_table(['Hash tags','Count'],pop_hashtag,10)



+----------------------+-------+
|    User mentions     | Count |
+----------------------+-------+
|   Hillary Clinton    |  6621 |
|   Face The Nation    |  1298 |
|   Donald J. Trump    |  359  |
|    John Dickerson    |  321  |
|    Bernie Sanders    |  248  |
|   President Obama    |  213  |
| D Wasserman Schultz  |  153  |
|     CNN Politics     |  142  |
|    Carly Fiorina     |  126  |
| Angela #1A #DefundPP |  122  |
+----------------------+-------+
+-----------------------+-------+
|       Hash tags       | Count |
+-----------------------+-------+
|     HillaryClinton    |  381  |
|      Hillary2016      |  226  |
|          tcot         |  132  |
|        Benghazi       |   93  |
|     FaceTheNation     |   85  |
|       UniteBlue       |   62  |
| Rockefellerinitiative |   62  |
|       Disclosure      |   60  |
|         Truth         |   60  |
|       603forHRC       |   57  |
+-----------------------+-------+


*------------------------

#Problem 3: Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers, such as "ladygaga".
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [27]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

#----------------------------------------------
# Define a function to make robust twitter requests
def make_twitter_request(twitter_api_func, max_errors=10, *args, **kw):
    def handle_twitter_http_error(e, wait_period=2, sleep_when_rate_limited=True):
        if wait_period > 3600: # Seconds
            print >> sys.stderr, 'Too many retries. Quitting.'
            raise e
        if e.e.code == 401:
            print >> sys.stderr, 'Encountered 401 Error (Not Authorized)'
            return None
        elif e.e.code == 404:
            print >> sys.stderr, 'Encountered 404 Error (Not Found)'
            return None
        elif e.e.code == 429:
            print >> sys.stderr, 'Encountered 429 Error (Rate Limit Exceeded)'
            if sleep_when_rate_limited:
                print >> sys.stderr, "Retrying in 15 minutes...ZzZ..."
                sys.stderr.flush()
                time.sleep(60*15 + 5)
                print >> sys.stderr, '...ZzZ...Awake now and trying again.'
                return 2
            else:
                raise e # Caller must handle the rate limiting issue
        elif e.e.code in (500, 502, 503, 504):
            print >> sys.stderr, 'Encountered %i Error. Retrying in %i seconds' % \
            (e.e.code, wait_period)
            time.sleep(wait_period)
            wait_period *= 1.5
            return wait_period
        else:
            raise e 
    wait_period = 2
    error_count = 0
    while True:
        try:
            return twitter_api_func(*args, **kw)
        except twitter.api.TwitterHTTPError, e:
            error_count = 0
            wait_period = handle_twitter_http_error(e, wait_period)
            if wait_period is None:
                return
        except URLError, e:
            error_count += 1
            print >> sys.stderr, "URLError encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise
        except BadStatusLine, e:
            error_count += 1
            print >> sys.stderr, "BadStatusLine encountered. Continuing."
            if error_count > max_errors:
                print >> sys.stderr, "Too many consecutive errors...bailing out."
                raise

#----------------------------------------------
# Define a function to get friend_ids and follower_ids
def get_friends_followers(twitter_api, screen_name=None, user_id=None,
                          friends_limit=maxint, followers_limit=maxint):
    assert(screen_name !=None) != (user_id != None),"Must have screen_name or user_id, but not both"
    # partial fetch for ids is set to 5000
    get_friends_ids = partial(make_twitter_request, twitter_api.friends.ids,
                              count=5000) 
    get_followers_ids = partial(make_twitter_request, twitter_api.followers.ids,
                                count=5000)
    friends_ids, followers_ids = [], []
    for twitter_api_func, limit, ids, label in [
        [get_friends_ids, friends_limit, friends_ids, "friends"], 
        [get_followers_ids, followers_limit, followers_ids, "followers"]]:
        if limit == 0:continue
        cursor = -1
        while cursor !=0:
            # if screen_name is provided
            if screen_name:
                response = twitter_api_func(screen_name = screen_name, cursor = cursor)
            # if user_id is provided
            else:
                response = twitter_api_func(user_id=user_id, cursor=cursor)
            if response is not None:
                ids += response['ids'] 
                cursor = response['next_cursor']
            print >> sys.stderr, 'Fetched {0} total {1} ids for {2}'.format(len(ids),label, (user_id or screen_name))
            # break when num of ids exceeds limit
            if len(ids) >= limit or response is None:
                break
    return friends_ids[:friends_limit],followers_ids[:followers_limit]

# getting the first 20 friends and followers of HC
friends_ids, followers_ids = get_friends_followers(twitter_api,
screen_name="HillaryClinton",
friends_limit=20,
followers_limit=20)
friends = [[p["id"],p["name"]] for p in make_twitter_request(twitter_api.users.lookup,user_id = friends_ids)]
make_table(['Friends_ID','Name'],friends,20)
followers = [[p["id"],p["name"]] for p in make_twitter_request(twitter_api.users.lookup,user_id = followers_ids)]
make_table(['Followers_ID','Name'],followers,20)

Fetched 247 total friends ids for HillaryClinton
Fetched 5000 total followers ids for HillaryClinton

+------------+---------------------+
| Friends_ID |         Name        |
+------------+---------------------+
|  15886631  |     Mike Kaiser     |
| 149972205  |   Ley Lin Chin ⭐️   |
| 3405913271 |      Cam Newton     |
| 3165917534 |  Brandon Tyler Cox  |
| 401771836  |     Ross Svenson    |
| 574239384  |    Laura O'Neill    |
| 427654338  |     Liz Mueller     |
| 3303293865 |    Caitlyn Jenner   |
|  23613479  |     Kris Jenner     |
| 169686021  |      KANYE WEST     |
|  25365536  | Kim Kardashian West |
| 103769342  |  Gabriella Cascone  |
| 2439027667 |     Jenny Glass     |
| 415823582  |      Sam Powers     |
| 161527155  |      Kevin Groh     |
| 3062962055 |     Sara Marino     |
| 3280722818 |       Pavitra       |
|  33992481  |   Catherine Crooks  |
+------------+---------------------+
+--------------+---------------------+
| Followers_ID |         Name        |
+--------------+---------------------+
|  1462999573  |   chichirodriguez   |
|  3632454554  |   Mary Ann Or

* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [1]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
#----------------------------------------------
# Define a function to get the mutual friends
def mutual(screen_name=None, user_id=None):
    friends_ids,followers_ids = get_friends_followers(twitter_api,screen_name=screen_name,followers_limit = 50000)
    mutual_ids = set(friends_ids).intersection(set(followers_ids))
    ls = [[person_id,twitter_api.users.show(user_id = person_id)["name"]] for person_id in mutual_ids]
    return ls
r = mutual(screen_name="HillaryClinton")
make_table(["ID","Name"],r)





Fetched 249 total friends ids for HillaryClinton
Fetched 5000 total followers ids for HillaryClinton
Fetched 10000 total followers ids for HillaryClinton
Fetched 15000 total followers ids for HillaryClinton
Fetched 20000 total followers ids for HillaryClinton
Fetched 25000 total followers ids for HillaryClinton
Fetched 30000 total followers ids for HillaryClinton
Fetched 35000 total followers ids for HillaryClinton
Fetched 40000 total followers ids for HillaryClinton
Fetched 45000 total followers ids for HillaryClinton
Fetched 50000 total followers ids for HillaryClinton

+------------+-------------+
|     ID     |     Name    |
+------------+-------------+
| 3245135844 | Huma Abedin |
+------------+-------------+


*------------------------

#Problem 4: (optional) Explore the data 

Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API

In [37]:
#Analysis 1: Comparing the supportive rates of three candidates
# Getting tweets about other two candidates
Donald_Trump = twitter_search(twitter_api, "DonaldTrump")
Bernie_Sanders = twitter_search(twitter_api, "BernieSanders")
save_json("DonaldTrump", results)
save_json("BernieSanders", results)
print len(Donald_Trump)
print len(Bernie_Sanders)


8100
8100


In [93]:
#Word Count for these candidates
make_table(['Words','Count'],Counter(word_process(statuses)).most_common()[:100],100) #Hillary Clinton's
make_table(['Words','Count'],Counter(word_process(Donald_Trump)).most_common()[:100],100) # Donad Trump's
make_table(['Words','Count'],Counter(word_process(Bernie_Sanders)).most_common()[:100],100) #Bernie Sander's

+------------------------+-------+
|         Words          | Count |
+------------------------+-------+
|          that          |  597  |
|         &amp;          |  508  |
|         about          |  492  |
|          with          |  408  |
|          this          |  407  |
|         woman          |  394  |
|          says          |  383  |
|          more          |  344  |
|          have          |  342  |
|        Hillary         |  311  |
|          your          |  271  |
|          than          |  255  |
|          from          |  242  |
|          real          |  239  |
|         being          |  224  |
|         first          |  211  |
|         should         |  206  |
|          just          |  202  |
|          will          |  201  |
|          like          |  192  |
|          what          |  177  |
|        campaign        |  174  |
|          Says          |  171  |
|         would          |  170  |
|        outsider        |  165  |
|          they     

In [88]:
def basic_analysis(topictweets):
    # This Regex captures all retweets using "RT & via"
    rt_patterns = re.compile(r"(RT|via)((?:\b\W*@\w+)+)", re.IGNORECASE)
    # This Regex filters hashtags, user mentions and urls
    text_pattern = re.compile(r'(^|\s)[^\w]*[#@][^\s]*|http[s]*://[^\s]*',flags = re.M) 
    s1 = [s for s in topictweets if rt_patterns.match(s['text']) == None] 
    s2 = [s for s in topictweets if rt_patterns.match(s['text']) != None]
    ss= set()
    rt_s = []
    # tweets without RT & via
    nrt_s = [[s['retweet_count'],s['user']['name'],text_pattern.sub('',s['text'])] for s in s1]
    # tweets with via & RT. Handles duplicates
    for s in s2:
        text = text_pattern.sub('',s['text'])
        if text not in ss:
            rt_s +=[[s['retweet_count'],s['user']['name'],text_pattern.sub('',s['text'])]]
            ss.add(text)
        else:
            continue
    return [nrt_s,rt_s]
# get the top 100 tweets of Hillary Clinton      
sorted_statuses_rt = sorted(basic_analysis(statuses)[1],reverse=True)[:100]
make_table(['Count','Name','Texts'],sorted_statuses_rt,100)
sorted_statuses_nrt = sorted(basic_analysis(statuses)[0],reverse=True)[:100]
make_table(['Count','Name','Texts'],sorted_statuses_nrt,100)





+-------+----------------------+----------------------------------------------------------------------------------------------------------------------------------+
| Texts |        Count         |                                                               Name                                                               |
+-------+----------------------+----------------------------------------------------------------------------------------------------------------------------------+
|  4232 |      Jewel Lee       |                   RT Before you watch her on my show today, watch her Whip. And of course, watch her Nae Nae.                    |
|  2818 |    indieactivity     |                                                    RT Retweet if you agree.                                                      |
|  2751 |      Glenn Coco      |                                                    RT is on my NYC premiere.                                                     |
|  2546 |      S

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10 minutes' talk) to present about the case study . We will ask two teams which are randomly selected to present their case studies in class for this case study. 

* ** Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through myWPI, in the Assignment "Case Study 1".
        
** Note: Each team just need to submit one submission in myWPI **

# Grading Criteria:

** Totoal Points: 100 **


---------------------------------------------------------------------------
** Notebook:  **
    Points: 60


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 

    -----------------------------------
    -----------------------------------
    Problem 4 (Optional): Explore the data
    Bonus points: 10
        Novelty: 5
        Interestingness: 5
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API




---------------------------------------------------------------------------
** Report: communicate the results**
    Points: 20

(1) What data you collected?
    Points: 5 

(2) Why this topic is interesting or important to you? (Motivations)
    Points: 5 

(3) How did you analyse the data?
    Points: 5 

(4) What did you find in the data?
(please include figures or tables in the report, but no source code)
    Points: 5 



---------------------------------------------------------------------------
** Slides (for 10 minutes of presentation): Story-telling **
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

